In [3]:
import pandas as pd
import numpy as np
train_data = pd.read_csv('kaggle/data/train.csv')
test_data = pd.read_csv('kaggle/data/test.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
passenger_ids = test_data['PassengerId']

In [5]:
train_data['Age'] = train_data.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.mean()))
test_data['Age'] = test_data.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.mean()))
train_data['Fare'] = train_data.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.mean()))
test_data['Fare'] = test_data.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.mean()))

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [8]:
train_data['CabinBool'] = train_data['Cabin'].notnull().astype(int)


train_data['Grade'] = train_data['Cabin'].str[0]  
train_data['Grade'] = train_data['Grade'].fillna('Unknown')  

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
train_data['Grade'] = encoder.fit_transform(train_data['Grade'])

In [9]:
test_data['CabinBool'] = test_data['Cabin'].notnull().astype(int)


test_data['Grade'] = test_data['Cabin'].str[0]  
test_data['Grade'] = test_data['Grade'].fillna('Unknown')  

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
test_data['Grade'] = encoder.fit_transform(test_data['Grade'])

In [10]:
# 타이타닉 데이터 예시
# train_data에 'Age' 컬럼이 있다고 가정

# 나이가 14세 이하인 사람을 구별하는 새로운 변수 추가
train_data['Age_14_or_under'] = train_data['Age'].apply(lambda x: 1 if x <= 14 else 0)
test_data['Age_14_or_under'] = test_data['Age'].apply(lambda x: 1 if x <= 14 else 0)
# 결과 확인
print(train_data[['Age', 'Age_14_or_under']].head())

    Age  Age_14_or_under
0  22.0                0
1  38.0                0
2  26.0                0
3  35.0                0
4  35.0                0


In [11]:
# Fare 요금의 90번째 백분위수 계산
fare_90th_percentile = train_data['Fare'].quantile(0.9)
fare_90th_percentile_2 = test_data['Fare'].quantile(0.9)
# Fare가 90번째 백분위수 이상인 사람을 구별하는 새로운 변수 추가
train_data['Top_10_Percent_Fare'] = train_data['Fare'].apply(lambda x: 1 if x >= fare_90th_percentile else 0)
test_data['Top_10_Percent_Fare'] = test_data['Fare'].apply(lambda x: 1 if x >= fare_90th_percentile_2 else 0)

In [12]:
train_data['Top_10_Percent_Fare'].value_counts()

Top_10_Percent_Fare
0    801
1     90
Name: count, dtype: int64

In [13]:
# # train_data에서 'Age_group'을 더미 변수로 변환
# train_data['Age_group'] = pd.cut(train_data['Age'], 
#                                   bins=[0, 12, 18, 40, 60, 100], 
#                                   labels=['0-12', '13-18', '19-40', '41-60', '60+'])

# # test_data에도 'Age_group'을 더미 변수로 변환
# test_data['Age_group'] = pd.cut(test_data['Age'], 
#                                  bins=[0, 12, 18, 40, 60, 100], 
#                                  labels=['0-12', '13-18', '19-40', '41-60', '60+'])

# X, y 생성
features = ['Pclass', 'Sex', 'Top_10_Percent_Fare', 'Age_14_or_under', 'Age', 'Fare', 'Grade', 'CabinBool']


x = pd.get_dummies(train_data[features])
test_rf= pd.get_dummies(test_data[features])

# train_data의 'Survived'를 y로 설정
y = train_data['Survived']

In [141]:
x

,Pclass,Top_10_Percent_Fare,Age_14_or_under,Age,Fare,Grade,CabinBool,Sex_female,Sex_male
0,3,0,0,22.00000,7.2500,8,0,False,True
1,1,0,0,38.00000,71.2833,2,1,True,False
2,3,0,0,26.00000,7.9250,8,0,True,False
3,1,0,0,35.00000,53.1000,2,1,True,False
4,3,0,0,35.00000,8.0500,8,0,False,True
...,...,...,...,...,...,...,...,...,...
886,2,0,0,27.00000,13.0000,8,0,False,True
887,1,0,0,19.00000,30.0000,1,1,True,False
888,3,0,0,25.14062,23.4500,8,0,True,False
889,1,0,0,26.00000,30.0000,2,1,False,True


In [142]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          418 non-null    int64  
 1   Pclass               418 non-null    int64  
 2   Name                 418 non-null    object 
 3   Sex                  418 non-null    object 
 4   Age                  418 non-null    float64
 5   SibSp                418 non-null    int64  
 6   Parch                418 non-null    int64  
 7   Ticket               418 non-null    object 
 8   Fare                 418 non-null    float64
 9   Cabin                91 non-null     object 
 10  Embarked             418 non-null    object 
 11  CabinBool            418 non-null    int64  
 12  Grade                418 non-null    int64  
 13  Age_14_or_under      418 non-null    int64  
 14  Top_10_Percent_Fare  418 non-null    int64  
dtypes: float64(2), int64(8), object(5)
memor

In [143]:
# # embarked 추가
# # Embarked를 더미 변수로 변환
# embarked_dummies = pd.get_dummies(train_data['Embarked'], prefix='Embarked')

# # 기존 데이터에 더미 변수 추가
# x = pd.concat([x, embarked_dummies], axis=1)

# embarked_dummies = pd.get_dummies(test_data['Embarked'], prefix='Embarked')

# # 기존 데이터에 더미 변수 추가
# test_rf = pd.concat([test_rf, embarked_dummies], axis=1)


In [144]:
# import numpy as np
# from sklearn.model_selection import cross_validate
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import StratifiedKFold
# # 모델 정의
# rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1)

# # cross-validation 수행
# scores = cross_validate(rf, x, y, return_train_score=True, cv=StratifiedKFold())

# # train_score와 test_score의 평균을 각각 계산 후 출력
# mean_train_score = np.mean(scores['train_score'])
# mean_test_score = np.mean(scores['test_score'])

# # 두 값을 한 번에 출력
# print(f"Mean Train Score: {mean_train_score}")
# print(f"Mean Test Score: {mean_test_score}")

In [14]:
from sklearn.model_selection import cross_validate

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1)
rf.fit(x,y)
scores = cross_validate(rf, x, y, return_train_score=True)
mean_train_score = np.mean(scores['train_score'])
mean_test_score = np.mean(scores['test_score'])
print(f"Mean Train Score: {mean_train_score}")
print(f"Mean Test Score: {mean_test_score}")

Mean Train Score: 0.9436027546212395
Mean Test Score: 0.8260686711443098


In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [120, 130],         # 트리의 개수
    'max_depth': [10, 12],             # 트리의 최대 깊이
    'min_samples_split': [5, 10],         # 내부 노드를 분할하는 최소 샘플 수
    'min_samples_leaf': [2, 3],           # 리프 노드에 필요한 최소 샘플 수
    'max_features': ['log2', 'sqrt'], # 고려할 특성의 수
    'bootstrap': [True, False],              # 부트스트랩 샘플링 여부
    'random_state': [42, 0]               # 랜덤 시드 값
}

# 그리드 서치 객체 정의
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# 데이터 준비 (X, y)
# 예: X_train, y_train으로 데이터를 준비했다고 가정합니다.

# 그리드 서치 실행
grid_search.fit(x, y)

# 최적의 파라미터와 성능 출력
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 128 candidates, totalling 640 fits
Best Parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 120, 'random_state': 42}
Best Score: 0.83841566756638


In [28]:
rf = RandomForestClassifier(n_estimators=120, max_depth=10, random_state=42, max_features='log2', min_samples_split=5, min_samples_leaf=2)
rf.fit(x, y)
scores = cross_validate(rf, x, y, return_train_score=True)
mean_train_score = np.mean(scores['train_score'])
mean_test_score = np.mean(scores['test_score'])
print(f"Mean Train Score: {mean_train_score}")
print(f"Mean Test Score: {mean_test_score}")
predictions=rf.predict(test_rf)

Mean Train Score: 0.9003939675685897
Mean Test Score: 0.83841566756638


In [29]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [30]:
output = pd.DataFrame({'PassengerId': passenger_ids, 'Survived': predictions})
output.to_csv('submission2.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
